In [2]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from data_encoders import *
from parametrizations import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

### Generate data

In [89]:
backend = Aer.get_backend('qasm_simulator')
n_samples = 2
n_features = 2
n_targets = 1

x = np.linspace(0, 2, n_samples)
y = x**2
y = y - np.min(y)
y = y/np.max(y)
y = np.pi/2 *y
y = y.reshape(-1,1) + 0.5

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2
#X[:,2] = x**3
#X[:,3] = x**4



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

print(X)
print(y)

[[0.000 0.000]
 [3.142 3.142]]
[[0.500]
 [2.071]]


In [137]:
backend = Aer.get_backend('qasm_simulator')
n_samples = 2
n_features = 2
n_targets = 1

np.random.seed(41)
X = np.random.uniform(0, np.pi, (n_samples, n_features))
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=2,
                      backend=backend, 
                      shots=100000)
y = model.predict(X)
print(X)
print(y)

[[0.788 0.145]
 [2.126 0.137]]
[[0.468]
 [0.825]]


### Parallel Model

In [138]:
reps = 2
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

optimizer = Adam()
optimizer.initialize(model.theta.shape)

In [139]:
for i in tqdm(range(100)):
    grad = [model.gradient(X, y)]
    grad = optimizer(grad)
    model.theta += -0.05*grad[0]
    y_pred = model.predict(X)
    mse = np.mean((y_pred - y)**2)
    loss = model.loss(X,y)
    #print(loss)
    #print(loss, model.theta, y_pred.flatten())
    print(f"{loss=:.3f}, {mse=:.3f}, {y_pred.flatten()=}")

  0%|          | 0/100 [00:00<?, ?it/s]

loss=0.764, mse=0.057, y_pred.flatten()=array([0.798, 0.894])
loss=0.741, mse=0.048, y_pred.flatten()=array([0.775, 0.858])
loss=0.718, mse=0.041, y_pred.flatten()=array([0.754, 0.819])
loss=0.692, mse=0.036, y_pred.flatten()=array([0.733, 0.779])
loss=0.658, mse=0.034, y_pred.flatten()=array([0.713, 0.739])
loss=0.629, mse=0.034, y_pred.flatten()=array([0.696, 0.696])
loss=0.603, mse=0.037, y_pred.flatten()=array([0.679, 0.654])
loss=0.579, mse=0.042, y_pred.flatten()=array([0.665, 0.614])
loss=0.552, mse=0.050, y_pred.flatten()=array([0.659, 0.575])
loss=0.530, mse=0.058, y_pred.flatten()=array([0.652, 0.538])
loss=0.500, mse=0.068, y_pred.flatten()=array([0.653, 0.506])
loss=0.487, mse=0.079, y_pred.flatten()=array([0.652, 0.474])
loss=0.464, mse=0.090, y_pred.flatten()=array([0.656, 0.446])
loss=0.449, mse=0.101, y_pred.flatten()=array([0.664, 0.422])
loss=0.430, mse=0.110, y_pred.flatten()=array([0.668, 0.400])
loss=0.417, mse=0.122, y_pred.flatten()=array([0.676, 0.378])
loss=0.4

KeyboardInterrupt: 

### Single-Layer Quantum Neural Network

In [142]:
np.random.seed(42)
network = sequential_qnn(q_bits=[3],
                         dim=[3, 1],
                         reps = 1,
                         scale = [2*np.pi],
                         backend=backend,
                         shots=10000,
                         lr=0.05
                         )

In [143]:
network.train(X,y, epochs=50, verbose=True)

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0, loss: 8.193679674311664
epoch: 1, loss: 7.495601826956253
epoch: 2, loss: 7.020641342356033
epoch: 3, loss: 6.351164121565445
epoch: 4, loss: 6.0358710013610075
epoch: 5, loss: 5.520674330890984
epoch: 6, loss: 5.562985219362489
epoch: 7, loss: 5.226416685863533
epoch: 8, loss: 5.349069155446584
epoch: 9, loss: 4.969325299670549
epoch: 10, loss: 4.895931547157998
epoch: 11, loss: 4.560273496233378
epoch: 12, loss: 4.534577473470142
epoch: 13, loss: 4.308888074115077
epoch: 14, loss: 3.856226874240236
epoch: 15, loss: 3.6226704451300558
epoch: 16, loss: 3.142794779657863
epoch: 17, loss: 2.888011938870995
epoch: 18, loss: 2.5432682039411123
epoch: 19, loss: 2.2145696335878715
epoch: 20, loss: 2.0104928140897815
epoch: 21, loss: 1.650924968401179
epoch: 22, loss: 1.346955972949928
epoch: 23, loss: 1.1471010600716651
epoch: 24, loss: 0.9720810936042443
epoch: 25, loss: 0.8107246040375287
epoch: 26, loss: 0.6789408351184673
epoch: 27, loss: 0.5496966430160615
epoch: 28, loss: 0.4

In [62]:
print(network.predict(X).flatten())
print(y.flatten())

[1.352 0.863 1.935 2.091]
[1.340 0.767 1.800 1.855]


# Cheese

In [141]:
backend = Aer.get_backend('qasm_simulator')
shots = 1000000

register1 = qk.QuantumRegister(1)
register2 = qk.QuantumRegister(1)
register3 = qk.QuantumRegister(1)
ancilla = qk.QuantumRegister(1)
classical = qk.ClassicalRegister(1)
registers = [register1, register2, register3, ancilla, classical]

circuit = qk.QuantumCircuit(*registers)

circuit.ry(2, register2)
circuit.ry(2, register3)

circuit.h(ancilla)
circuit.cswap(ancilla, register2, register3)
circuit.h(ancilla)

circuit.measure(ancilla, classical)

job = qk.execute(circuit, backend, shots=shots)
counts = job.result().get_counts(circuit)
if "0" in counts:
    value = counts["0"] / shots
else:
    value = 0

print(circuit)
print(value)

                              
q45547_0: ────────────────────
          ┌───────┐           
q45548_0: ┤ RY(2) ├─X─────────
          ├───────┤ │         
q45549_0: ┤ RY(2) ├─X─────────
          └─┬───┬─┘ │ ┌───┐┌─┐
q45550_0: ──┤ H ├───■─┤ H ├┤M├
            └───┘     └───┘└╥┘
 c7128: 1/══════════════════╩═
                            0 
1.0


In [110]:
backend = Aer.get_backend('qasm_simulator')
shots = 100000

register1 = qk.QuantumRegister(1)
register2 = qk.QuantumRegister(1)
register3 = qk.QuantumRegister(1)
ancilla = qk.QuantumRegister(1)
classical = qk.ClassicalRegister(1)
registers = [register1, register2, register3, ancilla, classical]

circuit = qk.QuantumCircuit(*registers)

circuit.ry(np.pi/2, register1)
circuit.cx(register1, register2)
circuit.ry(np.pi/2, register3)

circuit.h(ancilla)
circuit.cswap(ancilla, register2, register3)
circuit.h(ancilla)

circuit.measure(ancilla, classical)

job = qk.execute(circuit, backend, shots=shots)
counts = job.result().get_counts(circuit)
if "0" in counts:
    value = counts["0"] / shots
else:
    value = 0

print(circuit)
print(value)

          ┌──────────┐                
q23768_0: ┤ RY(pi/2) ├──■─────────────
          └──────────┘┌─┴─┐           
q23769_0: ────────────┤ X ├─X─────────
          ┌──────────┐└───┘ │         
q23770_0: ┤ RY(pi/2) ├──────X─────────
          └──┬───┬───┘      │ ┌───┐┌─┐
q23771_0: ───┤ H ├──────────■─┤ H ├┤M├
             └───┘            └───┘└╥┘
 c3347: 1/══════════════════════════╩═
                                    0 
0.75118


In [70]:
print(circuit)

        ┌─────────┐                 
q464_0: ┤ RY(2pi) ├───■─────────────
        └─────────┘ ┌─┴─┐           
q465_0: ────────────┤ X ├─X─────────
        ┌──────────┐└───┘ │         
q466_0: ┤ RY(pi/2) ├──────X─────────
        └──┬───┬───┘      │ ┌───┐┌─┐
q467_0: ───┤ H ├──────────■─┤ H ├┤M├
           └───┘            └───┘└╥┘
 c46: 1/══════════════════════════╩═
                                  0 
